# 15 - Catalogo de componentes novos

Este notebook valida e demonstra o carregamento de **todos os componentes novos** adicionados na paridade GUI/backend.

Objetivo:
- mostrar o YAML minimo de cada componente;
- validar que o parser carrega sem `Unsupported component type`;
- conferir a contagem de dispositivos/virtuais por caso.


In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
_candidates = []
for _ in range(6):
    for _rel in (("build-test", "python"), ("build", "python")):
        candidate = _root / _rel[0] / _rel[1]
        if candidate.is_dir():
            _candidates.append(candidate)
    _root = _root.parent

_seen = set()
for candidate in _candidates:
    cstr = str(candidate)
    if cstr in _seen:
        continue
    _seen.add(cstr)
    if cstr not in sys.path:
        sys.path.insert(0, cstr)

import pulsim as ps
import numpy as np
import matplotlib.pyplot as plt

print(f"Pulsim version: {ps.__version__}")


Pulsim version: 0.3.3


In [2]:
import textwrap

COMPONENT_SNIPPETS = {
    "BJT_NPN": """
- type: BJT_NPN
  name: QN1
  nodes: [base, in, out]
  beta: 80
""",
    "BJT_PNP": """
- type: BJT_PNP
  name: QP1
  nodes: [base, in, out]
  beta: 80
""",
    "THYRISTOR": """
- type: THYRISTOR
  name: SCR1
  nodes: [gate, in, 0]
  gate_threshold: 1.0
  holding_current: 0.05
  latch_current: 0.1
""",
    "TRIAC": """
- type: TRIAC
  name: TRI1
  nodes: [gate, in, 0]
  gate_threshold: 1.0
  holding_current: 0.05
  latch_current: 0.1
""",
    "SWITCH": """
- type: SWITCH
  name: S1
  nodes: [in, sw]
  g_on: 1e4
  g_off: 1e-9
""",
    "FUSE": """
- type: FUSE
  name: F1
  nodes: [in, fuse_out]
  rating: 5
  blow_i2t: 2
""",
    "CIRCUIT_BREAKER": """
- type: CIRCUIT_BREAKER
  name: B1
  nodes: [in, breaker_out]
  trip_current: 5
  trip_time: 1e-3
""",
    "RELAY": """
- type: RELAY
  name: K1
  nodes: [coil_p, coil_n, com, no, nc]
  pickup_voltage: 6
  dropout_voltage: 3
  contact_resistance: 0.05
  off_resistance: 1e9
""",
    "OP_AMP": """
- type: OP_AMP
  name: A1
  nodes: [in, out, ctrl]
  open_loop_gain: 1e5
  rail_low: -12
  rail_high: 12
""",
    "COMPARATOR": """
- type: COMPARATOR
  name: CMP1
  nodes: [in, out, cmp_out]
  threshold: 0.0
  hysteresis: 0.2
  high: 5.0
  low: 0.0
""",
    "PI_CONTROLLER": """
- type: PI_CONTROLLER
  name: PI1
  nodes: [in, out, pi_out]
  kp: 1.0
  ki: 10.0
  output_min: 0.0
  output_max: 1.0
""",
    "PID_CONTROLLER": """
- type: PID_CONTROLLER
  name: PID1
  nodes: [in, out, pid_out]
  kp: 1.0
  ki: 10.0
  kd: 0.1
  output_min: 0.0
  output_max: 1.0
""",
    "MATH_BLOCK": """
- type: MATH_BLOCK
  name: MATH1
  nodes: [in, out]
  operation: add
""",
    "PWM_GENERATOR": """
- type: PWM_GENERATOR
  name: PWM1
  nodes: [ctrl]
  frequency: 10000
  duty: 0.4
""",
    "INTEGRATOR": """
- type: INTEGRATOR
  name: INT1
  nodes: [in, out]
  output_min: -2.0
  output_max: 2.0
""",
    "DIFFERENTIATOR": """
- type: DIFFERENTIATOR
  name: DIFF1
  nodes: [in, out]
  alpha: 0.5
""",
    "LIMITER": """
- type: LIMITER
  name: LIM1
  nodes: [in, out]
  min: -1.0
  max: 1.0
""",
    "RATE_LIMITER": """
- type: RATE_LIMITER
  name: RL1
  nodes: [in, out]
  rising_rate: 10
  falling_rate: 10
""",
    "HYSTERESIS": """
- type: HYSTERESIS
  name: HYS1
  nodes: [in, out]
  threshold: 0.0
  hysteresis: 0.5
  high: 1.0
  low: -1.0
""",
    "LOOKUP_TABLE": """
- type: LOOKUP_TABLE
  name: LUT1
  nodes: [in, out]
  x: [0, 1, 2]
  y: [0, 10, 20]
""",
    "TRANSFER_FUNCTION": """
- type: TRANSFER_FUNCTION
  name: TF1
  nodes: [in, out]
  num: [0.5, 0.5]
  den: [1.0, -0.5]
""",
    "DELAY_BLOCK": """
- type: DELAY_BLOCK
  name: DEL1
  nodes: [in, out]
  delay: 1e-6
""",
    "SAMPLE_HOLD": """
- type: SAMPLE_HOLD
  name: SH1
  nodes: [in, out]
  sample_period: 1e-6
""",
    "STATE_MACHINE": """
- type: STATE_MACHINE
  name: SM1
  nodes: [ctrl]
  mode: toggle
  threshold: 0.5
""",
    "SATURABLE_INDUCTOR": """
- type: SATURABLE_INDUCTOR
  name: LSAT1
  nodes: [in, sat]
  inductance: 1m
  saturation_current: 2
  saturation_inductance: 100u
""",
    "COUPLED_INDUCTOR": """
- type: COUPLED_INDUCTOR
  name: K1
  nodes: [p1, p2, s1, s2]
  l1: 1m
  l2: 2m
  coupling: 0.95
""",
    "SNUBBER_RC": """
- type: SNUBBER_RC
  name: SN1
  nodes: [in, out]
  resistance: 100
  capacitance: 1n
""",
    "VOLTAGE_PROBE": """
- type: VOLTAGE_PROBE
  name: VP1
  nodes: [out, 0]
""",
    "CURRENT_PROBE": """
- type: CURRENT_PROBE
  name: IP1
  nodes: [in, 0]
  target_component: V1
""",
    "POWER_PROBE": """
- type: POWER_PROBE
  name: PP1
  nodes: [in, 0]
  target_component: V1
""",
    "ELECTRICAL_SCOPE": """
- type: ELECTRICAL_SCOPE
  name: SCOPE_E
  nodes: [in, out, 0]
""",
    "THERMAL_SCOPE": """
- type: THERMAL_SCOPE
  name: SCOPE_T
  nodes: [in]
""",
    "SIGNAL_MUX": """
- type: SIGNAL_MUX
  name: MUX1
  nodes: [in, out, ctrl]
  select_index: 1
""",
    "SIGNAL_DEMUX": """
- type: SIGNAL_DEMUX
  name: DMX1
  nodes: [in, out_a, out_b]
""",
}

print(f"Total de componentes novos no catalogo: {len(COMPONENT_SNIPPETS)}")


Total de componentes novos no catalogo: 34


In [3]:
def build_yaml_case(component_entry: str) -> str:
    base = textwrap.dedent(
        """
        schema: pulsim-v1
        version: 1
        simulation:
          tstart: 0
          tstop: 2e-6
          dt: 1e-6
        components:
          - type: voltage_source
            name: V1
            nodes: [in, 0]
            waveform: {type: dc, value: 5}
          - type: resistor
            name: R1
            nodes: [in, out]
            value: 1k
        """
    ).strip()
    entry = textwrap.indent(textwrap.dedent(component_entry).strip(), "  ")
    return f"{base}\n{entry}\n"

rows = []
for component_name, component_entry in COMPONENT_SNIPPETS.items():
    parser = ps.YamlParser()
    circuit, _ = parser.load_string(build_yaml_case(component_entry))
    ok = len(parser.errors) == 0
    rows.append(
        {
            "component": component_name,
            "status": "OK" if ok else "ERRO",
            "devices": circuit.num_devices() if ok else 0,
            "virtual": circuit.num_virtual_components() if ok else 0,
            "errors": " | ".join(parser.errors) if parser.errors else "",
        }
    )

print(f"{'COMPONENTE':<22} {'STATUS':<8} {'DEV':>4} {'VIRT':>5}")
print("-" * 50)
for row in rows:
    print(f"{row['component']:<22} {row['status']:<8} {row['devices']:>4} {row['virtual']:>5}")

failed = [row for row in rows if row["status"] != "OK"]
print(f"\nTotal OK: {len(rows) - len(failed)} / {len(rows)}")
if failed:
    print("\nFalhas:")
    for row in failed:
        print(f"- {row['component']}: {row['errors']}")

assert not failed, "Um ou mais componentes nao foram carregados."


COMPONENTE             STATUS    DEV  VIRT
--------------------------------------------------
BJT_NPN                OK          3     0
BJT_PNP                OK          3     0
THYRISTOR              OK          3     1
TRIAC                  OK          3     1
SWITCH                 OK          3     0
FUSE                   OK          3     1
CIRCUIT_BREAKER        OK          3     1
RELAY                  OK          4     1
OP_AMP                 OK          2     1
COMPARATOR             OK          2     1
PI_CONTROLLER          OK          2     1
PID_CONTROLLER         OK          2     1
MATH_BLOCK             OK          2     1
PWM_GENERATOR          OK          2     1
INTEGRATOR             OK          2     1
DIFFERENTIATOR         OK          2     1
LIMITER                OK          2     1
RATE_LIMITER           OK          2     1
HYSTERESIS             OK          2     1
LOOKUP_TABLE           OK          2     1
TRANSFER_FUNCTION      OK          2     1
DEL

## Resultado esperado

Se tudo estiver correto, a celula acima deve terminar com `Total OK: ...` e sem falhas.
